# Data Preparation for GeSAI - AB Data Challenge 

In [22]:
# Import necessary libraries
import pandas as pd
import dask.dataframe as dd
import numpy as np
import gc

In [ ]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import holidays
import gc

# --- 0. Configuración General ---
MAIN_PARQUET = '../data/official-data/data_ab3_complete.parquet'
OUTPUT_PARQUET_DIR = '../data/processed-data/dataset_FINAL_COMPLETO/'
NUMERO_DE_TROZOS = 60
LLAVE_DISTRITO = 'KEY_DISTRITO'
LLAVE_SECCION = 'KEY_SECCION'


RENTA_CSV = '../data/open-data/renda_procesada.csv'
ANTIGUITAD_CSV = '../data/open-data/antiguitat_pivotada.csv'
POBLACION_CSV = '../data/open-data/poblacion_pivotada.csv'
OBRES_CSV = '../data/open-data/obres_procesadas.csv'
# --- FIN DE LA ACTUALIZACIÓN ---

# --- 1. Cargar y Preparar Tablas Pequeñas (con PANDAS) ---
print("Cargando 4 datasets geográficos...")
dtype_llaves = {LLAVE_DISTRITO: 'string', LLAVE_SECCION: 'string'}
df_renta = pd.read_csv(RENTA_CSV, dtype=dtype_llaves)
df_antiguitat = pd.read_csv(ANTIGUITAD_CSV, dtype=dtype_llaves)
df_poblacion = pd.read_csv(POBLACION_CSV, dtype=dtype_llaves)
df_obres = pd.read_csv(OBRES_CSV, dtype=dtype_llaves)

print("Uniendo archivos geográficos...")
df_geo_total = df_renta.merge(df_antiguitat, on=[LLAVE_DISTRITO, LLAVE_SECCION], how='outer')
df_geo_total = df_geo_total.merge(df_poblacion, on=[LLAVE_DISTRITO, LLAVE_SECCION], how='outer')
df_geo_total = df_geo_total.merge(df_obres, on=[LLAVE_DISTRITO, LLAVE_SECCION], how='outer')
nuevas_columnas_geo = df_geo_total.columns.drop([LLAVE_DISTRITO, LLAVE_SECCION]).tolist()
df_geo_total = df_geo_total.fillna(0)
print(f"Tabla de características GEO creada con {len(df_geo_total)} filas.")

# 1B: Preparar datos METEO (Del Notebook)
print("Cargando datos de AEMET...")
df_aemet1 = pd.read_json('../data/open-data/data_aemet_1.json')
df_aemet2 = pd.read_json('../data/open-data/data_aemet_2.json')
df_aemet = pd.concat([df_aemet1, df_aemet2], ignore_index=True)
df_aemet = df_aemet[['fecha', 'tmed', 'tmin', 'tmax', 'prec', 'hrMedia']]
df_aemet.fillna({'prec': 0}, inplace=True)
cols_to_rename = {
    'fecha': 'FECHA', 'tmed': 'TEMP_MEDIA', 'tmin': 'TEMP_MIN',
    'tmax': 'TEMP_MAX', 'prec': 'PRECIPITACION', 'hrMedia': 'HUMEDAD_RELATIVA_MEDIA'
}
df_aemet.rename(columns=cols_to_rename, inplace=True)
df_aemet['FECHA'] = pd.to_datetime(df_aemet['FECHA'], errors='coerce')
nuevas_columnas_meteo = list(cols_to_rename.values())[1:]

# 1C: Preparar datos FESTIVOS (Del Notebook)
print("Generando datos de Festivos...")
fechas = pd.date_range(start='2024-01-01', end='2024-12-31')
df_fechas = pd.DataFrame({'FECHA': fechas})
es_holidays = holidays.CountryHoliday('ES', subdiv='CT', years=2024)
df_fechas['FESTIVO'] = df_fechas['FECHA'].apply(lambda date: date in es_holidays)
def get_tipo_dia_simple(fecha, es_festivo):
    if es_festivo: return 'Festivo'
    elif fecha.weekday() >= 5: return 'Fin de Semana'
    else: return 'Laborable'
df_fechas['TIPO_DIA'] = df_fechas.apply(lambda row: get_tipo_dia_simple(row['FECHA'], row['FESTIVO']), axis=1)
df_fechas['FECHA'] = pd.to_datetime(df_fechas['FECHA'], errors='coerce')
nuevas_columnas_festivos = ['FESTIVO', 'TIPO_DIA']

# 1D: Unir tablas pequeñas de Meteo y Festivos
df_meteo_festivos = pd.merge(df_aemet, df_fechas, on='FECHA', how='outer')

# --- 2. Cargar y Preparar Tabla Gigante (con DASK) ---
print("Cargando archivo principal (76M filas) con DASK...")
df_main_dask = dd.read_parquet(MAIN_PARQUET)
print(f"Creando {NUMERO_DE_TROZOS} trozos más pequeños...")
df_main_dask = df_main_dask.repartition(npartitions=NUMERO_DE_TROZOS)

# 2A: Limpiar SECCIO_CENSAL
df_main_dask = df_main_dask.dropna(subset=['SECCIO_CENSAL'])

# 2B: Imputar CONSUMO_REAL
df_main_dask['CONSUMO_REAL'] = df_main_dask['CONSUMO_REAL'].fillna(0)

# 2C: Crear Columnas FUGA (Manejando nulos)
def crear_columnas_fuga(df_particion):
    df_particion['FUGA_DETECTADA'] = df_particion['CODIGO_MENSAJE'].apply(
        lambda x: 0 if pd.isna(x) else (1 if x in ['FUITA', 'REITERACIÓ DE FUITA'] else 0)
    )
    df_particion['FUGA_REITERADA'] = df_particion['CODIGO_MENSAJE'].apply(
        lambda x: 0 if pd.isna(x) else (1 if x == 'REITERACIÓ DE FUITA' else 0)
    )
    return df_particion

print("Creando columnas de Fuga...")
new_meta = df_main_dask._meta.copy()
new_meta['FUGA_DETECTADA'] = pd.Series(dtype='int64')
new_meta['FUGA_REITERADA'] = pd.Series(dtype='int64')
df_main_dask = df_main_dask.map_partitions(crear_columnas_fuga, meta=new_meta)
df_main_dask = df_main_dask.drop(columns=['CREATED_MENSAJE', 'CODIGO_MENSAJE', 'TIPO_MENSAJE'])

# 2D: Separar FECHA_HORA
print("Separando Fecha y Hora...")
df_main_dask['FECHA'] = df_main_dask['FECHA_HORA'].dt.date
df_main_dask['HORA'] = df_main_dask['FECHA_HORA'].dt.time.astype(str) # Corrección V12
df_main_dask['FECHA'] = dd.to_datetime(df_main_dask['FECHA'], errors='coerce')

# --- 3. MERGES FINALES (con DASK) ---
print("Preparando Merge 1 (Meteo + Festivos)...")
df_main_dask = dd.merge(
    df_main_dask,
    df_meteo_festivos,
    on='FECHA',
    how='left'
)

print("Preparando Merge 2 (Geográfico)...")
llave_str = df_main_dask['SECCIO_CENSAL'].astype('string')
df_main_dask[LLAVE_DISTRITO] = llave_str.str.slice(5, 7).astype('string')
df_main_dask[LLAVE_SECCION] = llave_str.str.slice(7, 10).astype('string')

df_final_dask = dd.merge(
    df_main_dask,
    df_geo_total,
    on=[LLAVE_DISTRITO, LLAVE_SECCION],
    how='left'
)

# --- 4. Limpieza y Guardado Final ---
del df_renta, df_antiguitat, df_poblacion, df_obres, df_geo_total, df_aemet, df_fechas, df_meteo_festivos
gc.collect()

print("Limpiando nulos finales...")
fill_values_geo = {col: 0 for col in nuevas_columnas_geo}
fill_values_meteo = {col: 0 for col in nuevas_columnas_meteo}
fill_values_festivos = {'FESTIVO': False, 'TIPO_DIA': 'Laborable'} 

df_final_dask = df_final_dask.fillna(value=fill_values_geo)
df_final_dask = df_final_dask.fillna(value=fill_values_meteo)
df_final_dask = df_final_dask.fillna(value=fill_values_festivos)
df_final_dask['FESTIVO'] = df_final_dask['FESTIVO'].astype(bool)

# --- 5. Ejecutar y Guardado ---
print("¡EJECUTANDO! Dask está procesando TODO y guardando en disco...")
df_final_dask.to_parquet(OUTPUT_PARQUET_DIR, write_index=False)

print(f"¡HECHO! Tu dataset final y completo está en la CARPETA: '{OUTPUT_PARQUET_DIR}'")

Cargando 4 datasets geográficos...
Uniendo archivos geográficos...
Tabla de características GEO creada con 1069 filas.
Cargando datos de AEMET...
Generando datos de Festivos...
Cargando archivo principal (76M filas) con DASK...
Creando 60 trozos más pequeños...
Creando columnas de Fuga...
Separando Fecha y Hora...
Preparando Merge 1 (Meteo + Festivos)...
Preparando Merge 2 (Geográfico)...


c:\Users\barco\anaconda3\Lib\site-packages\dask\dataframe\multi.py:169: UserWarning: Merging dataframes with merge column data type mismatches: 
+----------------------------------+------------+-------------+
| Merge columns                    | left dtype | right dtype |
+----------------------------------+------------+-------------+
| ('KEY_DISTRITO', 'KEY_DISTRITO') | string     | string      |
| ('KEY_SECCION', 'KEY_SECCION')   | string     | string      |
+----------------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


Limpiando nulos finales...
¡EJECUTANDO! Dask está procesando TODO y guardando en disco...
¡HECHO! Tu dataset final y completo está en la CARPETA: 'dataset_FINAL_COMPLETO/'


In [1]:
# --- EN TU NOTEBOOK 1 (ETL con Dask) ---

# ... (después de todos los merges y limpiezas) ...

print("Eliminando duplicados finales...")

# 1. Definimos qué constituye un duplicado "real"
# Un cliente no debería tener dos filas para la misma hora exacta.
subset_duplicados = ['POLISSA_SUBM', 'FECHA', 'HORA']

# 2. Eliminamos duplicados en Dask
# (Dask necesita saber cómo ordenar para decidir cuál quedarse, 
#  por defecto se queda con el primero de la partición)
df_final_dask = df_final_dask.drop_duplicates(subset=subset_duplicados)

# --- Exportación ---
print("5. INICIANDO CÓMPUTO FINAL y Guardado...")
df_final_dask.to_parquet(OUTPUT_PARQUET_DIR, write_index=False)

Eliminando duplicados finales...


NameError: name 'df_final_dask' is not defined